# Training a classfier

In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [10]:
# transforms.Normalize?

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize( (0.5))]
)


SyntaxError: invalid syntax (<ipython-input-10-008670fd13e2>, line 5)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize?
)
tranforms.Normalize

In [ ]:
transform = transforms.Compose